In [1]:
from sklearn.ensemble import GradientBoostingClassifier
import pickle
from sklearn.metrics import ndcg_score
import pandas as pd
import gc

In [2]:
# training set
x = pd.read_csv("data/dummy/train.csv")

# val set 20%
x_val = x.sample(n=int(0.1*len(x)))
x = x.drop(x_val.index)

## TODO: date_time
x.drop(['click_bool', 'booking_bool', 'date_time'], axis=1, inplace=True)
x_val.drop(['click_bool', 'booking_bool', 'date_time'], axis=1, inplace=True)

# sort both on search id
x.sort_values('srch_id', ignore_index=True)
x_val.sort_values('srch_id', ignore_index=True)

,srch_id,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,...,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,orig_destination_distance,random_bool,agg_comp_rate,agg_comp_perc,agg_comp_inv,target
0,1,12,187,219,95166,3,4.0,1,2.71,0.0359,...,4,0,1,1,-1.00,1,0.0,9.0,0.000000,0
1,1,12,187,219,10404,4,4.0,1,2.20,0.0149,...,4,0,1,1,-1.00,1,0.0,0.0,0.333333,0
2,1,12,187,219,111106,3,2.5,1,0.69,0.0071,...,4,0,1,1,-1.00,1,0.0,0.0,0.000000,0
3,4,5,219,219,127808,2,2.5,0,3.00,0.0457,...,1,0,1,1,237.73,1,0.0,0.0,0.000000,0
4,4,5,219,219,137826,2,3.0,0,3.26,0.3147,...,1,0,1,1,238.21,1,0.0,0.0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495829,332782,19,158,158,95592,4,0.0,0,1.61,0.0000,...,1,0,1,1,-1.00,0,0.0,0.0,0.000000,0
495830,332784,24,219,100,51585,4,4.5,0,4.67,0.0258,...,1,0,1,0,329.34,0,0.0,0.0,0.000000,0
495831,332785,5,219,219,94508,3,3.5,1,1.10,0.0164,...,3,0,1,0,544.43,0,0.0,0.0,0.000000,0
495832,332785,5,219,219,55110,3,1.0,1,2.20,0.0000,...,3,0,1,0,512.78,0,0.0,0.0,0.000000,0


In [3]:
y_val = x_val.target.squeeze()
y = x.target.squeeze()
x.drop(['target'], axis=1, inplace=True)
x_val.drop(['target'], axis=1, inplace=True)

In [4]:
groups = x.srch_id.value_counts().sort_index()
groups_val = x_val.srch_id.value_counts().sort_index()

In [5]:
import lightgbm

# default lightgbm model with sklearn api
gbm = lightgbm.LGBMRanker(metric="ndcg", objective="lambdarank", num_boost_round=1000) 

#label_gain

##rank_xendcg, XE_NDCG_MART ranking objective function, aliases: xendcg, xe_ndcg, xe_ndcg_mart, xendcg_mart
#rank_xendcg is faster than and achieves the similar performance as lambdarank

# fitting model 
gbm.fit(
    x,
    y,
    eval_set = [(x_val, y_val)],
    group = groups,
    eval_group = [groups_val],
    verbose=1,
    early_stopping_rounds=100
)

/home/susy/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] num_iterations is set=1000, num_boost_round=1000 will be ignored. Current value: num_iterations=1000
[1]	valid_0's ndcg@1: 0.940517	valid_0's ndcg@2: 0.958931	valid_0's ndcg@3: 0.96713	valid_0's ndcg@4: 0.970669	valid_0's ndcg@5: 0.97209
Training until validation scores don't improve for 100 rounds
[2]	valid_0's ndcg@1: 0.943143	valid_0's ndcg@2: 0.961084	valid_0's ndcg@3: 0.968894	valid_0's ndcg@4: 0.972249	valid_0's ndcg@5: 0.973588
[3]	valid_0's ndcg@1: 0.944231	valid_0's ndcg@2: 0.961906	valid_0's ndcg@3: 0.969524	valid_0's ndcg@4: 0.972797	valid_0's ndcg@5: 0.974115
[4]	valid_0's ndcg@1: 0.944728	valid_0's ndcg@2: 0.962159	valid_0's ndcg@3: 0.969805	valid_0's ndcg@4: 0.973037	valid_0's ndcg@5: 0.974326
[5]	valid_0's ndcg@1: 0.945163	valid_0's ndcg@2: 0.96257	valid_0's ndcg@3: 0.97006	valid_0's ndcg@4: 0.973318	valid_0's ndcg@5: 0.97457
[6]	valid_0's ndcg@1: 0.945089	valid_0's ndcg@2: 0.962628	valid_0's ndcg@3: 0.970124	valid_0's ndcg@4: 0.97331	valid_0's ndcg@

[60]	valid_0's ndcg@1: 0.946052	valid_0's ndcg@2: 0.963433	valid_0's ndcg@3: 0.970722	valid_0's ndcg@4: 0.973788	valid_0's ndcg@5: 0.975033
[61]	valid_0's ndcg@1: 0.946024	valid_0's ndcg@2: 0.963429	valid_0's ndcg@3: 0.970703	valid_0's ndcg@4: 0.973783	valid_0's ndcg@5: 0.975027
[62]	valid_0's ndcg@1: 0.946103	valid_0's ndcg@2: 0.963485	valid_0's ndcg@3: 0.970783	valid_0's ndcg@4: 0.973815	valid_0's ndcg@5: 0.975069
[63]	valid_0's ndcg@1: 0.94612	valid_0's ndcg@2: 0.963466	valid_0's ndcg@3: 0.970791	valid_0's ndcg@4: 0.973805	valid_0's ndcg@5: 0.975073
[64]	valid_0's ndcg@1: 0.946166	valid_0's ndcg@2: 0.963514	valid_0's ndcg@3: 0.970807	valid_0's ndcg@4: 0.973832	valid_0's ndcg@5: 0.975094
[65]	valid_0's ndcg@1: 0.946132	valid_0's ndcg@2: 0.963459	valid_0's ndcg@3: 0.970764	valid_0's ndcg@4: 0.973814	valid_0's ndcg@5: 0.975073
[66]	valid_0's ndcg@1: 0.94612	valid_0's ndcg@2: 0.963492	valid_0's ndcg@3: 0.970771	valid_0's ndcg@4: 0.973817	valid_0's ndcg@5: 0.975076
[67]	valid_0's ndcg@1:

[120]	valid_0's ndcg@1: 0.94568	valid_0's ndcg@2: 0.963155	valid_0's ndcg@3: 0.97054	valid_0's ndcg@4: 0.973583	valid_0's ndcg@5: 0.974864
[121]	valid_0's ndcg@1: 0.945675	valid_0's ndcg@2: 0.963147	valid_0's ndcg@3: 0.970533	valid_0's ndcg@4: 0.973582	valid_0's ndcg@5: 0.974856
[122]	valid_0's ndcg@1: 0.945635	valid_0's ndcg@2: 0.96313	valid_0's ndcg@3: 0.970534	valid_0's ndcg@4: 0.973567	valid_0's ndcg@5: 0.974843
[123]	valid_0's ndcg@1: 0.945664	valid_0's ndcg@2: 0.963137	valid_0's ndcg@3: 0.97055	valid_0's ndcg@4: 0.973575	valid_0's ndcg@5: 0.974854
[124]	valid_0's ndcg@1: 0.945618	valid_0's ndcg@2: 0.963122	valid_0's ndcg@3: 0.970531	valid_0's ndcg@4: 0.973562	valid_0's ndcg@5: 0.974839
[125]	valid_0's ndcg@1: 0.945692	valid_0's ndcg@2: 0.963155	valid_0's ndcg@3: 0.970547	valid_0's ndcg@4: 0.973585	valid_0's ndcg@5: 0.974864
[126]	valid_0's ndcg@1: 0.945692	valid_0's ndcg@2: 0.963144	valid_0's ndcg@3: 0.970542	valid_0's ndcg@4: 0.973588	valid_0's ndcg@5: 0.974863
[127]	valid_0's n

LGBMRanker(metric='ndcg', num_boost_round=1000, objective='lambdarank')

In [6]:
gc.collect()
with open('model/model', 'wb') as f:
    pickle.dump(gbm, f)

In [7]:
with open('model/model', 'rb') as f:
    model = pickle.load(f)

In [8]:
test = pd.read_csv("data/dummy/test.csv", index_col=False)
test.drop('date_time', axis=1, inplace=True)

In [9]:
predictions = model.predict(test)

In [10]:
len(predictions)

4959183

In [11]:
submission = test[["srch_id", "prop_id"]]
submission['pred'] = predictions

<ipython-input-11-f49d9f7448f2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['pred'] = predictions


In [12]:
del test
gc.collect()

0

In [13]:
submission = submission.sort_values(['srch_id', 'pred'], ascending=[True, False])

In [14]:
submission.head(n=20)

,srch_id,prop_id,pred
23,1,99484,1.007238
9,1,54937,0.716831
12,1,61934,0.596628
5,1,28181,0.461306
4,1,24194,0.443674
8,1,50162,0.245483
18,1,82231,0.165198
20,1,90385,0.142966
6,1,34263,0.140413
13,1,63894,0.133516


In [15]:
submission[['srch_id', 'prop_id']].to_csv('submission.csv', index=False)